In [143]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [144]:
env = gym.make("FrozenLake-v1", is_slippery=True, map_name="4x4")

In [145]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


# EPSILON GREEDY STRATEGY
- When <font color = magenta>epsilon = 1</font>, it is 100% certain that the agent will __explore__ the environment.
> - As the agent learns more about the environment, <font color = magenta>epsilon</font> will decrease, and the agent will move from __exploring__ to __exploiting__.
> > - The rate at which <font color = magenta>epsilon</font> decays is the parameter we call <font color = cyan>exploration_decay_rate</font>.
> 

In [146]:
num_episodes = 10000 # total number of episodes for the agent to play during training
max_steps_per_episode = 1000 # maximum number of steps for agent to take within an episode

learning_rate = 0.9 # alpha in the eqn. after alpha = 0.5, learning rate seems to have a negative effect on rewards.
discount_rate = 0.99 # gamma in the eqn.

exploration_rate = 1 
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001 # a smaller decay rate results in the agent getting to rewards quicker.

In [147]:
rewards_all_episodes = []

# Q-learning algorithm
for episode in range(num_episodes):
    state = env.reset() # reset game.

    done = False
    rewards_current_episode = 0 # game hasn't begun yet, so no rewards.

    for step in range(max_steps_per_episode):

        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0,1)
        # if the exploration_rate_threshold is greater than the exploration_rate, the agent will
        #   EXPLOIT the environment, and choose the action that has the highest value in the Q table
        #   for the current state. otherwise, the agent will explore the environment and sample 
        #   an action randomly.
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()

        new_state, reward, done, info = env.step(action)

        # Update Q-table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state,:]))

        state = new_state
        rewards_current_episode += reward

        if done == True:
            break

    # Exploration rate decay
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)

    rewards_all_episodes.append(rewards_current_episode)

# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000
print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

# Print updated Q table.
print("\n\n********Q-table********\n")
print(q_table)

********Average reward per thousand episodes********

1000 :  0.02100000000000001
2000 :  0.09100000000000007
3000 :  0.1360000000000001
4000 :  0.2890000000000002
5000 :  0.36300000000000027
6000 :  0.45100000000000035
7000 :  0.44500000000000034
8000 :  0.44300000000000034
9000 :  0.4070000000000003
10000 :  0.44200000000000034


********Q-table********

[[6.28577484e-01 1.89533646e-01 3.55895447e-01 1.90024047e-01]
 [2.17329144e-05 6.24023232e-03 2.33264728e-02 6.37598593e-01]
 [5.55625742e-02 2.56833799e-02 3.42260011e-02 5.82124472e-01]
 [3.02324093e-05 2.89077843e-02 1.97308225e-02 2.42467823e-01]
 [8.40676351e-01 2.54232851e-03 5.05231764e-02 2.73046400e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.47603927e-01 6.73656211e-06 2.69702768e-06 1.36033364e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.06592368e-02 4.08335725e-03 9.39996649e-03 8.73042919e-01]
 [2.30926882e-02 1.89151856e-01 4.99049806e-04 7.11101183e-03]
 [1.3306394

In [148]:
'''for episode in range(5):
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)

    for step in range(max_steps_per_episode):
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)

        action = np.argmax(q_table[state,:])
        new_state, reward, done, info = env.step(action)

        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole :(****")
                time.sleep(3)
            clear_output(wait=True)
            break

        state = new_state

env.close()'''

'for episode in range(5):\n    state = env.reset()\n    done = False\n    print("*****EPISODE ", episode+1, "*****\n\n\n\n")\n    time.sleep(1)\n\n    for step in range(max_steps_per_episode):\n        clear_output(wait=True)\n        env.render()\n        time.sleep(0.3)\n\n        action = np.argmax(q_table[state,:])\n        new_state, reward, done, info = env.step(action)\n\n        if done:\n            clear_output(wait=True)\n            env.render()\n            if reward == 1:\n                print("****You reached the goal!****")\n                time.sleep(3)\n            else:\n                print("****You fell through a hole :(****")\n                time.sleep(3)\n            clear_output(wait=True)\n            break\n\n        state = new_state\n\nenv.close()'